In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math

Начало работы с файлом CSV

Выведем общие данные о таблице

In [2]:
df = pd.read_csv('Dataset1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   color                       5025 non-null   object 
 1   Director_Name               4872 non-null   object 
 2   num_Critic_for_reviews      4927 non-null   float64
 3   duration                    4959 non-null   float64
 4   director_Facebook_likes     4872 non-null   float64
 5   actor_3_Facebook_likes      4953 non-null   float64
 6   actor_2_name                4963 non-null   object 
 7   Actor_1_Facebook_likes      4968 non-null   float64
 8   gross                       4104 non-null   float64
 9   genres                      4974 non-null   object 
 10  actor_1_name                4968 non-null   object 
 11  movie_Title                 4974 non-null   object 
 12  num_voted_users             4974 non-null   float64
 13   cast_total_facebook_likes  4974 

Узнаем названия столбцов

In [3]:
df.columns

Index(['color', 'Director_Name', 'num_Critic_for_reviews', 'duration',
       'director_Facebook_likes', 'actor_3_Facebook_likes', 'actor_2_name',
       'Actor_1_Facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_Title', 'num_voted_users', ' cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes;'],
      dtype='object')

Узнаем количество пустых ячеек

In [4]:
df.isna().sum()

color                          18
Director_Name                 171
num_Critic_for_reviews        116
duration                       84
director_Facebook_likes       171
actor_3_Facebook_likes         90
actor_2_name                   80
Actor_1_Facebook_likes         75
gross                         939
genres                         69
actor_1_name                   75
movie_Title                    69
num_voted_users                69
 cast_total_facebook_likes     69
actor_3_name                   90
facenumber_in_poster           82
plot_keywords                 216
movie_imdb_link                69
num_user_for_reviews           87
language                       82
country                        72
content_rating                366
budget                        549
title_year                    174
actor_2_facebook_likes         80
imdb_score                     69
aspect_ratio                  389
movie_facebook_likes;          69
dtype: int64

Скорректируем названия коллон изаполним некоторые пустые ячейки

In [5]:
df = df.rename (
    columns = {
        ' cast_total_facebook_likes' : 'cast_total_facebook_likes',
        'Director_Name'              : 'director_name',
        'num_Critic_for_reviews'     : 'num_critic_for_reviews',
        'Actor_1_Facebook_likes'     : 'actor_1_facebook_likes',
        'director_Facebook_likes'    : 'director_facebook_likes',
        'actor_3_Facebook_likes'     : 'actor_3_facebook_likes',
        'movie_Title'                : 'movie_title'
    }
)

In [6]:
columns_to_replace =['color', 'director_name', 'actor_2_name', 'genres', 'actor_1_name', 'movie_title', 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'language', 'country', 'aspect_ratio']

for row in columns_to_replace:
    df[row] = df[row].fillna('unknown')
df.isna().sum()

color                          0
director_name                  0
num_critic_for_reviews       116
duration                      84
director_facebook_likes      171
actor_3_facebook_likes        90
actor_2_name                   0
actor_1_facebook_likes        75
gross                        939
genres                         0
actor_1_name                   0
movie_title                    0
num_voted_users               69
cast_total_facebook_likes     69
actor_3_name                   0
facenumber_in_poster          82
plot_keywords                  0
movie_imdb_link                0
num_user_for_reviews          87
language                       0
country                        0
content_rating               366
budget                       549
title_year                   174
actor_2_facebook_likes        80
imdb_score                    69
aspect_ratio                   0
movie_facebook_likes;         69
dtype: int64

Проверим наличие дубликатов

In [7]:
df.duplicated().sum()

45

Удалим дубликаты

In [8]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

В данной БД была замечена следующая аномалия: в некоторых строках вся информация содержится в столбце 'Color', поэтому разобьем эту ячейку на нужные нам 28 и добавим их в виде отдельной строки в нашу БД, а изначальные строки с такими ячейками 'Color' удалим.

In [9]:
movies_table = df
movies_table.info()
index = 0

for element in movies_table['color']:
    if ',' in element:
        title = element.split('"')[1]
        element = element.replace('"'+title+'"', '')
        list_element = element.split(",")
        list_element[11] = '"'+title+'"'
        
        df2 = pd.DataFrame([list_element], columns = movies_table.columns)
        movies_table = pd.concat([movies_table, df2])
        
for element in movies_table['color']:
    if ',' in element:       
        movies_table = movies_table.drop(labels = [index],axis = 0)
    index += 1
        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      4998 non-null   object 
 1   director_name              4998 non-null   object 
 2   num_critic_for_reviews     4884 non-null   float64
 3   duration                   4915 non-null   float64
 4   director_facebook_likes    4829 non-null   float64
 5   actor_3_facebook_likes     4909 non-null   float64
 6   actor_2_name               4998 non-null   object 
 7   actor_1_facebook_likes     4924 non-null   float64
 8   gross                      4070 non-null   float64
 9   genres                     4998 non-null   object 
 10  actor_1_name               4998 non-null   object 
 11  movie_title                4998 non-null   object 
 12  num_voted_users            4930 non-null   float64
 13  cast_total_facebook_likes  4930 non-null   float

Проверим остались ли эти ячейки

In [10]:
n = 0
for element in movies_table['color']:
    if ',' in element:
        n+=1
print(n)
movies_table.info()

0
<class 'pandas.core.frame.DataFrame'>
Index: 4998 entries, 0 to 0
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   color                      4998 non-null   object
 1   director_name              4998 non-null   object
 2   num_critic_for_reviews     4952 non-null   object
 3   duration                   4983 non-null   object
 4   director_facebook_likes    4897 non-null   object
 5   actor_3_facebook_likes     4977 non-null   object
 6   actor_2_name               4998 non-null   object
 7   actor_1_facebook_likes     4992 non-null   object
 8   gross                      4138 non-null   object
 9   genres                     4998 non-null   object
 10  actor_1_name               4998 non-null   object
 11  movie_title                4998 non-null   object
 12  num_voted_users            4998 non-null   object
 13  cast_total_facebook_likes  4998 non-null   object
 14  actor_3_name  

Начнем вычисления CPI

In [11]:
url = "http://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/"

r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, 'html.parser')

table = soup.find('table')
rows = table.tbody.findAll('tr')

years = []
cpis = []

for row in rows:
    year = row.findAll('td')[0].get_text()
    if year.isdigit() and int(year) < 2017 :
        years.append(int(year))
        cpis.append(float(row.findAll('td')[13].get_text()))

cpi_table = pd.DataFrame({
    "year": years,
    "avg_annual_cpi": cpis
})

cpi_table.head()

,year,avg_annual_cpi
0,1913,9.9
1,1914,10.0
2,1915,10.1
3,1916,10.9
4,1917,12.8


In [12]:
def get_real_value(nominal_amt, old_cpi, new_cpi):
    try:
        real_value = (nominal_amt * new_cpi) / old_cpi
        return real_value
    except TypeError:
        return float('nan')

In [13]:
CPI_2016 = float ( cpi_table [ cpi_table [ 'year' ] == 2016 ][ 'avg_annual_cpi' ])

C:\Users\я\AppData\Local\Temp\ipykernel_121888\3640390386.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  CPI_2016 = float ( cpi_table [ cpi_table [ 'year' ] == 2016 ][ 'avg_annual_cpi' ])


In [14]:
real_domestic_gross = []
real_budget_values = []

for index,row in movies_table.iterrows():
    gross = row['gross']
    budget = row['budget']
    year = row['title_year']
    try:
        cpi = float(cpi_table[cpi_table['year'] == int(year)]['avg_annual_cpi'])
    except:
        float('nan')
        
    real_gross = get_real_value(gross, cpi, CPI_2016)
    real_budget = get_real_value(budget, cpi, CPI_2016)
    real_domestic_gross.append(real_gross)
    real_budget_values.append(real_budget)

movies_table["real_domestic_gross"] = real_domestic_gross
movies_table["real_budget"] = real_budget_values

C:\Users\я\AppData\Local\Temp\ipykernel_121888\2193515267.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  cpi = float(cpi_table[cpi_table['year'] == int(year)]['avg_annual_cpi'])


Посчитаем окупаемость

In [15]:
profits = []
roi_vals = []

for index, row in movies_table.iterrows():
    profit = row['real_domestic_gross'] - row['real_budget']
    budget = row['real_budget']
    num = profit - budget
    den = budget
    roi = (num / den) *100

    profits.append(profit)
    roi_vals.append(roi)

movies_table['profit'] = profits
movies_table['roi'] = roi_vals

movies_table.head(10)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes;,real_domestic_gross,real_budget,profit,roi
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,237000000.0,2009.0,936.0,7.9,1.78,33000;,8.507937e+08,2.651368e+08,5.856569e+08,120.888543
1,Colour,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,300000000.0,2007.0,5000.0,7.1,2.35,0;,3.582208e+08,3.473329e+08,1.088790e+07,-96.865283
2,Colour,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,245000000.0,2015.0,393.0,6.8,2.35,85000;,2.025981e+08,2.480907e+08,-4.549257e+07,-118.337071
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,250000000.0,2012.0,23000.0,8.5,2.35,164000;,4.684551e+08,2.613385e+08,2.071167e+08,-20.747743
4,unknown,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,12.0,7.1,unknown,0;,NaN,NaN,NaN,NaN
5,Colour,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,263700000.0,2012.0,632.0,6.6,2.35,24000;,7.637218e+07,2.756598e+08,-1.992877e+08,-172.294775
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,258000000.0,2007.0,11000.0,6.2,2.35,0;,3.896268e+08,2.987063e+08,9.092051e+07,-69.561898
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,260000000.0,2010.0,553.0,7.8,1.85,29000;,2.210219e+08,2.861734e+08,-6.515148e+07,-122.766438
8,Colour,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,250000000.0,2015.0,21000.0,7.5,2.35,118000;,4.647818e+08,2.531538e+08,2.116281e+08,-16.403360
9,Colour,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,250000000.0,2009.0,11000.0,7.5,2.35,10000;,3.378055e+08,2.796802e+08,5.812535e+07,-79.217208


In [172]:
movies_table.to_csv(r'C:\Users\я\Desktop\ВУЗ\Мусияк.csv')

# *Гипотеза №1: Чем больше бюджет - тем больше коэффициент окупаемости проекта.*

In [16]:
movies_table_budget_roi = movies_table
movies_table_budget_roi = movies_table_budget_roi[['real_budget', 'roi']].sort_values(['real_budget'], ascending=False)

movies_table_budget_roi.head(10)

,real_budget,roi
2964,1.454269e+10,-199.981979
3830,5.161441e+09,-199.994960
2309,3.588890e+09,-199.904242
2981,3.072286e+09,-199.992164
2320,2.703122e+09,-199.980710
3397,2.231680e+09,-199.960076
4506,1.440618e+09,-198.996261
3822,1.030705e+09,-199.758956
3051,8.333576e+08,-199.532080
3248,6.604001e+08,-199.732922


In [17]:
movies_table_roi_budget = movies_table
movies_table_roi_budget = movies_table_roi_budget[['roi', 'real_budget']].sort_values(['roi'], ascending=False)

movies_table_roi_budget.head(10)

,roi,real_budget
4753,719248.553333,1.736664e+04
4759,271366.055046,2.843561e+02
4668,234016.856667,8.643709e+04
4939,40786.400000,3.937123e+04
4894,36742.728535,4.066585e+05
3253,36742.728535,4.066585e+05
4990,28956.000000,1.197469e+04
4911,22557.819000,1.012615e+05
4932,17537.489231,8.258579e+04
2477,15466.666667,1.104327e+06


Как мы видим гипотеза не подтвердилась, так как фильмы с самым большим бюджетом уступают фильмам с меньшим бюджетом в коэффициенте окупаемости проекта.

# *Гипотеза 2: фильмов жанра Drama с imdb рейтингом >7.5 больше нежели таких же фильомв в жанре Action*

In [144]:
#Создадим df состоящий только из фильмов жанра Drama
movies_table_drama = movies_table.loc [movies_table.loc[:, 'genres'] == 'Drama']
movies_table_drama

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes;,real_domestic_gross,real_budget,profit,roi
600,Color,Oliver Stone,297.0,136.0,0.0,17.0,Austin Pendleton,903.0,52474616.0,Drama,...,70000000.0,2010.0,592.0,6.3,2.35,13000;,5.775707e+07,7.704668e+07,-1.928961e+07,-125.036263
661,Color,Mike Newell,141.0,117.0,179.0,403.0,Kirsten Dunst,8000.0,63695760.0,Drama,...,65000000.0,2003.0,4000.0,6.4,1.85,0;,8.308385e+07,8.478508e+07,-1.701232e+06,-102.006523
675,Color,David Fincher,315.0,151.0,21000.0,637.0,Meat Loaf,11000.0,37023395.0,Drama,...,63000000.0,1999.0,783.0,8.8,2.35,48000;,5.333658e+07,9.075895e+07,-3.742237e+07,-141.232706
743,Color,Steven Zaillian,110.0,115.0,234.0,521.0,Kathleen Quinlan,3000.0,56702901.0,Drama,...,60000000.0,1998.0,552.0,6.5,2.35,1000;,8.349137e+07,8.834613e+07,-4.854766e+06,-105.495165
1138,Color,Gus Van Sant,137.0,136.0,835.0,225.0,Rob Brown,670.0,51768623.0,Drama,...,43000000.0,2000.0,370.0,7.3,2.35,0;,7.215350e+07,5.993206e+07,1.222143e+07,-79.607853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,Color,Tadeo Garcia,NaN,84.0,5.0,12.0,Michael Cortez,21.0,NaN,Drama,...,NaN,2004.0,20.0,6.1,unknown,22;,NaN,NaN,NaN,NaN
0,Color,James Bridges,28,107,21,418,Phoebe Cates,967,16118077,Drama,...,25000000,1988,767,5.6,1.85,377;,NaN,NaN,NaN,NaN
0,Color,Jorge Ramírez Suárez,26,120,45,69,Adriana Barraza,623,100412,Drama,...,4000000,2013,85,7.7,2.35,0;,NaN,NaN,NaN,NaN
0,Color,Steven Soderbergh,58,100,0,65,Laura San Giacomo,828,24741700,Drama,...,1200000,1989,531,7.2,1.85,2000;,NaN,NaN,NaN,NaN


In [145]:
#Теперь оставим только строки, в которых рейтинг >7.5
movies_table_drama[['imdb_score']] = movies_table_drama[['imdb_score']].astype(float)
movies_table_drama = movies_table_drama.loc [movies_table_drama['imdb_score'] > 7.5]
movies_table_drama.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 675 to 0
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      52 non-null     object 
 1   director_name              52 non-null     object 
 2   num_critic_for_reviews     52 non-null     object 
 3   duration                   52 non-null     object 
 4   director_facebook_likes    49 non-null     object 
 5   actor_3_facebook_likes     52 non-null     object 
 6   actor_2_name               52 non-null     object 
 7   actor_1_facebook_likes     52 non-null     object 
 8   gross                      42 non-null     object 
 9   genres                     52 non-null     object 
 10  actor_1_name               52 non-null     object 
 11  movie_title                52 non-null     object 
 12  num_voted_users            52 non-null     object 
 13  cast_total_facebook_likes  52 non-null     object 
 14  

C:\Users\я\AppData\Local\Temp\ipykernel_121888\3367820330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_table_drama[['imdb_score']] = movies_table_drama[['imdb_score']].astype(float)


In [146]:
#Повторяем все действия для второго жанра
movies_table_action = movies_table.loc [movies_table.loc[:, 'genres'] == 'Action']
movies_table_action

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes;,real_domestic_gross,real_budget,profit,roi
214,Color,Joel Schumacher,183.0,125.0,541.0,409.0,Vivica A. Fox,920.0,107285004.0,Action,...,125000000.0,1997.0,890.0,3.7,1.85,0;,1.604309e+08,1.869213e+08,-2.649049e+07,-114.171997
435,Color,Tim Burton,153.0,126.0,13000.0,390.0,Vincent Schiavelli,920.0,162831698.0,Action,...,80000000.0,1992.0,811.0,7.0,1.85,0;,2.785513e+08,1.368536e+08,1.416977e+08,3.539622
846,Black and White,Quentin Tarantino,354.0,111.0,16000.0,640.0,Vivica A. Fox,926.0,70098138.0,Action,...,30000000.0,2003.0,890.0,8.1,2.35,13000;,9.143502e+07,3.913158e+07,5.230344e+07,33.660460
2374,Color,Danny Pang,23.0,107.0,15.0,27.0,Angelica Lee,82.0,NaN,Action,...,NaN,2013.0,39.0,5.7,1.85,124;,NaN,NaN,NaN,NaN
2677,Color,RZA,208.0,107.0,561.0,353.0,RZA,746.0,15608545.0,Action,...,15000000.0,2012.0,561.0,5.4,2.35,29000;,1.631645e+07,1.568031e+07,6.361449e+05,-95.943033
2720,Color,Tony Jaa,110.0,110.0,0.0,7.0,Petchtai Wongkamlao,64.0,102055.0,Action,...,300000000.0,2008.0,45.0,6.2,2.35,0;,1.137649e+05,3.344222e+08,-3.343084e+08,-199.965982
2791,Color,Chao-Bin Su,50.0,117.0,2.0,21.0,Shawn Yue,149.0,NaN,Action,...,12000000.0,2010.0,32.0,6.9,2.35,1000;,NaN,1.320800e+07,NaN,NaN
2862,Color,Jon Hess,19.0,87.0,29.0,147.0,Ian Gomez,683.0,1200000.0,Action,...,3000000.0,1993.0,155.0,5.1,1.85,107;,1.993138e+06,4.982844e+06,-2.989707e+06,-160.000000
3407,Color,Michael Winnick,15.0,106.0,155.0,391.0,Helena Mattsson,981.0,NaN,Action,...,8000000.0,2016.0,505.0,4.2,unknown,689;,NaN,8.000000e+06,NaN,NaN
4373,unknown,John Stockwell,2.0,90.0,134.0,354.0,T.J. Storm,260000.0,NaN,Action,...,17000000.0,2016.0,454.0,9.1,unknown,0;,NaN,1.700000e+07,NaN,NaN


In [137]:
movies_table_action[['imdb_score']] = movies_table_action[['imdb_score']].astype(float)
movies_table_action = movies_table_action.loc [movies_table_action['imdb_score'] > 7.5]
movies_table_action.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 846 to 4373
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      2 non-null      object 
 1   director_name              2 non-null      object 
 2   num_critic_for_reviews     2 non-null      object 
 3   duration                   2 non-null      object 
 4   director_facebook_likes    2 non-null      object 
 5   actor_3_facebook_likes     2 non-null      object 
 6   actor_2_name               2 non-null      object 
 7   actor_1_facebook_likes     2 non-null      object 
 8   gross                      1 non-null      object 
 9   genres                     2 non-null      object 
 10  actor_1_name               2 non-null      object 
 11  movie_title                2 non-null      object 
 12  num_voted_users            2 non-null      object 
 13  cast_total_facebook_likes  2 non-null      object 
 14

C:\Users\я\AppData\Local\Temp\ipykernel_121888\3153453393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_table_action[['imdb_score']] = movies_table_action[['imdb_score']].astype(float)


Гипотеза полностью подтвердилась, фильмов жанра Drama с IMDB рэйтингом - 52, а жанра Action - 2.

# *Гипотеза 3: фильмы с наибольшим бюджетом имеют имеют продолжительность сеанса больше среднего значения*

In [168]:
#Возьмем топ-10 фильмов с наибольшим бюджетом
movies_table_topbudget = movies_table
movies_table_topbudget = movies_table[['real_budget', 'duration']].sort_values(['real_budget'], ascending=False)

movies_table_topbudget = movies_table_topbudget.head(10)
movies_table_topbudget

,real_budget,duration
2964,1.454269e+10,110.0
3830,5.161441e+09,112.0
2309,3.588890e+09,134.0
2981,3.072286e+09,134.0
2320,2.703122e+09,103.0
3397,2.231680e+09,124.0
4506,1.440618e+09,99.0
3822,1.030705e+09,115.0
3051,8.333576e+08,193.0
3248,6.604001e+08,90.0


In [164]:
#Посчитаем среднюю продолжительность среди фсех фильмов
test = movies_table
test[['duration']] = test[['duration']].astype(float)
avg_duration = test['duration'].mean()
print(avg_duration)

107.21332530604053


In [169]:
#Теперь проверим сколько из 10 наших фильмов по продолжительности дольше среднего значения
movies_table_topbudget[['duration']] = movies_table_topbudget[['duration']].astype(float)
movies_table_topbudget = movies_table_topbudget.loc [movies_table_topbudget['duration'] > avg_duration]
movies_table_topbudget

,real_budget,duration
2964,1.454269e+10,110.0
3830,5.161441e+09,112.0
2309,3.588890e+09,134.0
2981,3.072286e+09,134.0
3397,2.231680e+09,124.0
3822,1.030705e+09,115.0
3051,8.333576e+08,193.0


In [170]:
#Посчитаем разницу
print(10 - len(movies_table_topbudget.axes[0]))

3


7 из 10 фильмов с наибольшим бюджетом действительно идут дольше среднего времени, поэтому гипотезу можно засчитать за верную.

# *Выводы*

Были проверены три гипотезы:
    1. Зависимость коэффициента окупаемости фильма от его начального бюджета
    2. Количество фильмов жанра Drama и рейтингом imdb >7.5 больше фильмов жанра Action также с рейтингом >7.5
    3. Зависимость финального хронометража от начального бюджета

1) Первая гипотеза отвергается, такой зависимости не обнаружено.
2) Вторая гипотеза полностью подтвердилась, фильмов Drama с рейтингом imdb >7.5 больше чем фильмо Action с таким же условием рейтинга.
3) Третья гипотеза скорее подтвердилась, нежели опроверглась, чем больше бюджет фильма, тем больше его хронометраж.